In [ ]:
!pip install sentence-transformers faiss-cpu

In [ ]:
import google.generativeai as genai
from google.colab import userdata
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


In [1]:

class NormalRAG:
    def __init__(self, model_name='sentence-transformers/all-MiniLM-L6-v2', gemini_model_name='gemini-1.5-flash'):
        self.embedding_model = SentenceTransformer(model_name)
        self.gemini_model = genai.GenerativeModel(gemini_model_name)
        self.index = None
        self.documents = []

        # Configure Gemini API
        GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
        genai.configure(api_key=GOOGLE_API_KEY)

    def add_documents(self, docs):
        self.documents.extend(docs)
        embeddings = self.embedding_model.encode(docs)
        if self.index is None:
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(np.array(embeddings))

    def retrieve(self, query, k=3):
        query_embedding = self.embedding_model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding), k)
        return [self.documents[i] for i in indices[0]]

    def generate_response(self, query, retrieved_docs):
        context = "\n\n".join(retrieved_docs)
        prompt = f"Based on the following context, answer the query:\n\nContext:\n{context}\n\nQuery:\n{query}\n\nAnswer:"
        response = self.gemini_model.generate_content(prompt)
        return response.text

KeyboardInterrupt: 

**Explanation:**

1.  **`__init__`**: Initializes the `SentenceTransformer` for embeddings, the Gemini generative model, an empty FAISS index, and a list to store documents. It also configures the Gemini API using a key stored in Colab's secrets manager.
2.  **`add_documents`**: Takes a list of documents, adds them to the internal list, generates their embeddings using the `SentenceTransformer`, and adds these embeddings to the FAISS index.
3.  **`retrieve`**: Takes a query, generates its embedding, and uses the FAISS index to find the `k` most similar document embeddings, returning the corresponding documents.
4.  **`generate_response`**: Takes the original query and the retrieved documents, formats them into a prompt, and uses the Gemini model to generate a response based on the provided context.

**To use this class:**

1.  Make sure you have your Google API key stored as a secret named `GOOGLE_API_KEY` in Colab.
2.  Create an instance of the `NormalRAG` class.
3.  Use the `add_documents` method to add your text data.
4.  Use the `retrieve` method to get relevant documents for a query.
5.  Use the `generate_response` method to get an answer from the Gemini model based on the query and retrieved documents.